In [7]:
import os
from dotenv import find_dotenv, load_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI-API-KEY')
langchain_api_key = os.getenv('LANGCHAIN-API-KEY')

In [13]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

In [23]:
loader = WebBaseLoader(
    web_paths=("https://www.fcm.dk/billetter/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [25]:
print(f"Loaded {len(docs)} documents")
print(f"Number of splits: {len(splits)}") #No splits are made!
print(splits[:5])  # Print the first few splits to inspect their content


Loaded 1 documents
Number of splits: 0
[]


In [24]:
#Define vectorstore and retriever
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=openai_api_key))
retriever = vectorstore.as_retriever()

IndexError: list index out of range in upsert.